# Using RAG to Build a Custom ChatBot
## 4. Add-Ons / Improvements (Experiments)

> **Notice:**  
> Before starting this tutorial series, read up on the RAG pipeline.

This tutorial series assumes prerequisite understanding of RAG and therefore goes through the implementation of an advanced and customized RAG pipeline, explaining the micro-decisions made along the way.

> **Data Corpus:** 
> This tutorial uses [AGO yearly audit reports](https://www.ago.gov.sg/publications/annual-reports/) as an example. However, this repo's code is applicable to most pdf documents. The code examples for other documents (such as national day rally) will be referenced later. 

This notebook (Tutorial 4) will provide a summary of experiments conducted in trying to improve the RAG pipeline from Tutorial 3.

### Experiment 1: HyDE

Retrievers look for chunks similar to input query. However, answers need not be similar to questions. The HyDE technique overcomes this by 
1. generating a possible answer to the user's question
2. use the generated possible answer as the input query into retrievers to retrieve chunks similar to the hypothetical answer. 

Read more about HyDE [here]("https://medium.aiplanet.com/advanced-rag-improving-retrieval-using-hypothetical-document-embeddings-hyde-1421a8ec075a").

My HyDE experiments are documented in ["../experiments/hyde.ipynb"](../experiments/hyde.ipynb). I have found that it improves the final answer when specific years are queried for. Due to its partial success, all RAG notebooks mentioned provide the option of enabling HyDE (default disabled). 

If you find that your use case's output is improved by HyDE, feel free to enable HyDE by simply delaring ```HyDE = True```. One suggestion is to check with an agent if questions query for specific years and enabling HyDE dynamically. 

### Experiment 2: Prompt Compression

Due to ["Lost in the Middle"](https://arxiv.org/abs/2307.03172) effect, the prompts to LLMs cannot be too long. With only 10000 chunks, the average prompt can reach up to 5000 tokens. Datasets easily can go up to 1 million chunks, leading to prompts being a significant fraction of context windows of even GPT-4o. Therefore, we need to reduce the size of prompts without losing information. This is called prompt compression. 

Microsoft has developed models that can compress prompts, called the [LLMLingua series]("https://github.com/microsoft/LLMLingua"). LangChain offers LLMLingua and LlamaIndex offers LongLLMLingua. However, Microsoft recently released [LLMLingua-2]("https://www.microsoft.com/en-us/research/publication/llmlingua-2-data-distillation-for-efficient-and-faithful-task-agnostic-prompt-compression/"), the best compression model so far. All 3 models can also be directly accessed using Microsoft's provided API. 

I have carried out experiments on all 3 models and documented my results in ["../experiments/prompt_compression.ipynb"](../experiments/prompt_compression.ipynb).

### Experiment 3: Recursive Retrieval
#### *The promise of recursive retireval*
The first retireved chunks may not be best related to the question. This is due to the resons explained in HyDE. Therefore, a second retireval using chunks from the first retrieval as queries can be benificial. However, there are no open source solutions that carry out this form of retrieval yet. 

#### *LLamaIndex Recursive Retrieval*
Usually, a data corpus has multiple documents. LlamaIndex's version of recurive retrieval does retrieval first on the documents (maybe on the titles/descriptions of the documents), followed by a second retrieval on the documents. This may not be the best approach to retrieval if documents contain cross references.

For example, 2018's Audit report finds issues that have been present since 2011. Therefore, questions regarding 2011 should also check 2018's Audit report. However, recursive retrieval on questions regarding 2011 will only check 2011's Audit report. 
    
My experiments on LLamaIndex's implements recurive retrieval are documented in ["../experiments/recursive_retrieval.ipynb"](../experiments/recursive_retrieval.ipynb). Running this notebook will require another conda environment due to version conflicts with LlamaIndex not being updated to match LangChain. The instructions to set up this special environment is found in the README file. The version requirements are in ["../requirements_recursive_retrieval.txt"](../requirements_recursive_retrieval.txt)



### Experiment 4: Tables
My current pdf to text conversion does not take tables into account. However, the text in the tables and the surrounding context appear to be sufficient for the LLM to udnerstand contents in the table. Nevertheless, if you would like to add tables into the RAG pipeline, I have provided code (using LlamaIndex functions) in ["../experiments/tables.ipynb"](../experiments/tables.ipynb). This code works best on well formated tables such as scraped HTML but not very well on pdfs. 